In [1]:
import os
import glob
import json
import pickle
import numpy as np
import pandas as pd

import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

C:\Users\User\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [15]:
paths = [
    *glob.glob('C:/Users/User/Desktop/MIPT_Alpha/results/test*'),
    *glob.glob('C:/Users/User/Desktop/MIPT_Alpha/kaggle_checkpoints/test*'),
    *glob.glob('C:/Users/User/Desktop/MIPT_Alpha/colab_results/test*')
]

In [17]:
test_scores = []
for file_name in paths:
    with open(file_name, 'rb') as f:
        test_score = pickle.load(f)
        test_scores.append(test_score)

In [35]:
data_scores = torch.stack(test_scores).T
target = pd.read_csv('C:/Users/User/Desktop/MIPT_Alpha/data/train_target.csv', index_col=0)[-data_scores.shape[0]:]['flag'].to_numpy()

C:\Users\User\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [71]:
scores_dict = dict()
for i, score in enumerate(test_scores):
    scores_dict[paths[i][paths[i].find('\\') + 1:]] = roc_auc_score(target, score)
dict(sorted(scores_dict.items(), key=lambda x: x[1]))

{'test_kaggle_2_.pkl': 0.755813007017504,
 'test_colab_2.pkl': 0.7581369784436539,
 'test_kaggle_2.pkl': 0.7588360703600615,
 'test_kaggle_3.pkl': 0.7592574074117068,
 'test_kaggle_1_.pkl': 0.7605361520350746,
 'test_pc_1.pkl': 0.7605560073987289,
 'test_colab_1.pkl': 0.7607419405137302,
 'test_kaggle_1.pkl': 0.7618003578635422,
 'test_pc_2.pkl': 0.7619224018929069}

In [40]:
X_train, X_test, y_train, y_test = train_test_split(data_scores, target, shuffle=True, stratify=target)

In [72]:
y_train.sum() / y_train.size, y_test.sum() / y_test.size

(0.032456296296296296, 0.032462222222222226)

In [85]:
poly = PolynomialFeatures(degree=3, include_bias=False, interaction_only=True)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

In [88]:
gbc = GradientBoostingClassifier(max_depth=2)
gbc = gbc.fit(X_train_poly, y_train)
train_preds = gbc.predict_proba(X_train_poly)
test_preds = gbc.predict_proba(X_test_poly)

In [89]:
print('train',  roc_auc_score(y_train, train_preds[:, 1]))
print('test',  roc_auc_score(y_test, test_preds[:, 1]))

train 0.7704748068205702
test 0.7713819377573099


In [91]:
predict_paths = [
    *glob.glob('C:/Users/User/Desktop/MIPT_Alpha/results/predict*'),
    *glob.glob('C:/Users/User/Desktop/MIPT_Alpha/kaggle_checkpoints/predict*'),
    *glob.glob('C:/Users/User/Desktop/MIPT_Alpha/colab_results/predict*')
]

In [99]:
predict_scores = []
for file_name in predict_paths:
    with open(file_name, 'rb') as f:
        predict_score = pickle.load(f)
        predict_scores.append(predict_score)
        
predict_scores = torch.stack(predict_scores).T

In [103]:
X_predict_poly = poly.transform(predict_scores)
preds = gbc.predict_proba(X_predict_poly)[:, 1]

In [115]:
sample_submission = pd.read_csv('C:/Users/User/Desktop/MIPT_Alpha/sample_submission.csv')

In [107]:
sample_submission['score'] = preds

In [112]:
sample_submission.to_csv('C:/Users/User/Desktop/MIPT_Alpha/my_submission.csv', index=False)

In [113]:
my_submission = pd.read_csv('C:/Users/User/Desktop/MIPT_Alpha/my_submission.csv')

In [117]:
my_submission

,id,score
0,3000000,0.035091
1,3000001,0.033321
2,3000002,0.016121
3,3000003,0.023992
4,3000004,0.006596
...,...,...
499995,3499995,0.040455
499996,3499996,0.020900
499997,3499997,0.024475
499998,3499998,0.013526


*****************

{'test_kaggle_2_.pkl': 0.755813007017504,
 'test_colab_2.pkl': 0.7581369784436539,
 'test_kaggle_2.pkl': 0.7588360703600615,
 'test_kaggle_3.pkl': 0.7592574074117068,
 'test_kaggle_1_.pkl': 0.7605361520350746,
 'test_pc_1.pkl': 0.7605560073987289,
 'test_colab_1.pkl': 0.7607419405137302,
 'test_kaggle_1.pkl': 0.7618003578635422,
 'test_pc_2.pkl': 0.7619224018929069}

gbc = GradientBoostingClassifier(max_depth=2)

train 0.7695057180174542  
test 0.7710640248008458

*******************************

{'test_kaggle_2_.pkl': 0.755813007017504,
 'test_colab_2.pkl': 0.7581369784436539,
 'test_kaggle_2.pkl': 0.7588360703600615,
 'test_kaggle_3.pkl': 0.7592574074117068,
 'test_kaggle_1_.pkl': 0.7605361520350746,
 'test_pc_1.pkl': 0.7605560073987289,
 'test_colab_1.pkl': 0.7607419405137302,
 'test_kaggle_1.pkl': 0.7618003578635422,
 'test_pc_2.pkl': 0.7619224018929069}
 
 poly = PolynomialFeatures(degree=3, include_bias=False, interaction_only=True)  
 gbc = GradientBoostingClassifier(max_depth=2)
 
train 0.7704748068205702  
test 0.7713819377573099